In [1]:
import pandas as pd
import requests
import re
import time
session = requests.Session()

In [2]:
df = pd.read_pickle('../data/dune_rt_data.pkl')
df.head()

,rating,review,displayName,isVerified,isSuperReviewer,hasSpoilers,hasProfanity,score,timeFromCreation,user.displayName,user.accountLink,user.realm,user.userId,displayImageUrl
0,5.0,So I got to rewatch Dune again this time in a ...,Md T,False,False,False,False,5.0,7h ago,Md T,/user/id/979110234,RT,979110234,NaN
1,5.0,Movie was beautifully shot and acted. Sound de...,Rohan P,False,False,False,False,5.0,10h ago,Rohan P,/user/id/977222113,RT,977222113,NaN
2,5.0,Showing this magnificent book fron this side i...,Grzegorz B,False,False,False,False,5.0,11h ago,Grzegorz B,/user/id/979424531,RT,979424531,NaN
3,3.0,Beautiful. \nBut.\nI was predisposed to love t...,Susan S,False,False,False,False,3.0,16h ago,Susan S,/user/id/948447923,RT,948447923,NaN
4,5.0,"Blew my expectations out of the water, brillia...",kieran h,False,False,False,False,5.0,16h ago,kieran h,/user/id/979421884,RT,979421884,NaN


In [3]:
#subsetting for verified data because.. they're verified and makes things a bit simpler for computing and memory costs
verified_df = df[df['isVerified'] == True]
print(verified_df.shape)
verified_df.head()

(235, 14)


,rating,review,displayName,isVerified,isSuperReviewer,hasSpoilers,hasProfanity,score,timeFromCreation,user.displayName,user.accountLink,user.realm,user.userId,displayImageUrl
43,5.0,There is no finer rendition of this wonderful ...,Nathan S,True,False,False,False,5.0,3d ago,Nathan S,NaN,Fandango,8192C9FE-2BA1-4189-B677-E63733F47231,NaN
47,0.5,terrible. i wonder if they used cliff notes t...,Nathan P,True,False,False,False,0.5,3d ago,Nathan P,NaN,Fandango,4e7428cf-b2b2-4d98-948d-3e8fc792409a,NaN
85,3.0,"Reminded me of the last Jedi movie, only I rea...",jenn t,True,False,False,False,3.0,5d ago,jenn t,NaN,Fandango,5396A122-E93A-4100-A241-DDEBE3E1ECEB,NaN
94,2.5,Taking long to get going,Ernesto Gomez,True,False,False,False,2.5,6d ago,Ernesto Gomez,NaN,Fandango,359C959C-DDFF-4DD3-91F3-6E7F0B376F7B,NaN
120,3.5,One of the most visually stunning films ever. ...,W J W,True,False,False,False,3.5,"Dec 24, 2021",W J W,NaN,Fandango,1230EFFD-F1F2-43FE-90E9-5FDAC26B4162,NaN


In [4]:
# combining all reviews together
# FYI, there is no need to preprocess / clean the text with huggingface pipelines:
# https://blog.paperspace.com/natural-language-processing-with-huggingface/#:~:text=The%20pipeline%20contains%20the%20pre,to%20perform%20any%20text%20preprocessing
reviews = '. '.join([i for i in verified_df.review.values])
reviews[:1000]

"There is no finer rendition of this wonderful story, and heck, it even makes a decent film this time. Might not win many big time awards, but this is far better than fanclub pandering we get from Marvel or the LOTR films. This isn't somebody just paying with the characters, nor trying to recreate a book verbatim. This was myth retelling in its finest. Honor the heart of the characters and tell a story with the same meaning. In doing so Dune has been reborn and raking in new fans as it's reward. Will be exciting to see what happens next with this evolution.. terrible. i wonder  if they used cliff notes to write the script. i walked out, tried to wait to see a worm but couldnt stand it.. Reminded me of the last Jedi movie, only I really enjoyed that movie. This one was long and drawn out.. Taking long to get going. One of the most visually stunning films ever. Tough to adapt from the novel, but they did an impressive job. Just prepare yourself for this only being the first of two parts.

In [5]:
from transformers import pipeline

In [6]:
summarizer = pipeline("summarization", 
                      model="t5-base", 
                      tokenizer="t5-base", 
                       device = 0, # comment out line if there is no CUDA enabled GPU
)
summarizer.device

device(type='cuda', index=0)

In [7]:
len_char = 40000
input_text = reviews[:len_char] # gpu taps out around this length string 
print(f'Input text len: {len(input_text)} \n ')
summary_text = summarizer(input_text,
                          max_length=100,
                          min_length=5,
                          do_sample=False)[0]['summary_text']
print(summary_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (9129 > 512). Running this sequence through the model will result in indexing errors


Input text len: 37821 
 
the movie is a great start to a fantastic sci-fi story . it's great to see it on a big screen, but a little too long . i was disappointed with the '80s version of the book . this is the best movie i've seen in a decade .
